In [ ]:
import numpy as np
import pandas as pd


<h3> Ratings matrix shown below displays the ratings provided by users to different movies. </h3>

|  user_id  |  movie_id  |  rating  |  unix_timestamp  |
|:---       |   :---:    |  :---:   |             ---: |
|    196    |    242     |    3     |     881250949    |
|    186    |    302     |    3     |     891717742    |
|     22    |    377     |    1     |     878887116    |

In [ ]:
# Column headings of ratings given by users to different movies
rating_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('u.data', sep = '\t', names = rating_cols, encoding = 'latin-1')

print("Ratings details :")
print(ratings.head())

The movie matrix(1682 X 24) lists 1682 movies, each having 24 features.
This matrix is primarily used to extract the movie names corresponding to movie ids. 


As of now, the other features are reduntant since the feature vectors for movies are trained later.


In [ ]:
# Column headings of movies
movie_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep = '|', names = movie_cols, encoding = 'latin-1')

print("Movie details :")
print("shape : ", movies.shape)
print(movies.head())

The following user-movie matrix is created which is used in the training process.

Each row represents the movies rated by a user. If the user has watched/rated a movie, the cell value lies between 1(poorly rated) to 5(highly rated). In case a user has not watched a movie, the cell value is kept at 0.

|     | Toy Story (1995) | GoldenEye (1995) | Four Rooms (1995) | Get Shorty (1995) |
|:--- |       :---:      |       :---:      |        :---:      |              ---: |
|  1  |        5.0       |        3.0       |         4.0       |         3.0       |
|  2  |        4.0       |        0.0       |         0.0       |         0.0       |
|  3  |        0.0       |        0.0       |         0.0       |         0.0       |
|  4  |        0.0       |        0.0       |         0.0       |         0.0       |
|  5  |        4.0       |        3.0       |         0.0       |         0.0       |

In [ ]:
# Create the user-movie matrix
col_name = []
for row in movies.itertuples():
    col_name.append(row[2])

matrix_rows = ratings.user_id.unique().shape[0]
matrix_cols = ratings.movie_id.unique().shape[0]

ratings_matrix = np.zeros((matrix_rows, matrix_cols), dtype = np.int64)

# Filling the user-movie matrix
for row in ratings.itertuples():
    # row[1] : user_id, row[2] : movie_id, row[3] = rating
    ratings_matrix[row[1] - 1][row[2] - 1] = row[3]

ratings_matrix_df = pd.DataFrame(ratings_matrix, columns = col_name)

# converting numpy matrix to df
nan_value = float("NaN")
ratings_matrix_df = ratings_matrix_df.replace(0, nan_value)  
ratings_matrix_df = ratings_matrix_df.dropna(thresh = 10, axis = 1).fillna(0)

print(ratings_matrix_df.head())

For easier matrix operations, the dataframe is converted to a numpy array.

Our numpy array looks like:

<h3>[[5. 3. 4. ... 0. 0. 0.]</h3>

<h3> [4. 0. 0. ... 0. 0. 0.]</h3>

<h3> [0. 0. 0. ... 0. 0. 0.]</h3>

<h3> ... </h3>

<h3> [5. 0. 0. ... 0. 0. 0.]</h3>

<h3> [0. 0. 0. ... 0. 0. 0.]</h3>
 
<h3> [0. 5. 0. ... 0. 0. 0.]]</h3>

In [ ]:
# Convert to numpy for easier matrix operations
Ratings = ratings_matrix_df.to_numpy()
print(Ratings)

<h3>Create randomised feature vectors for movies and users.</h3>

In this case, both movie and user vectors are trained, so as to predict missing values(0 valued cells) in the user-move matrix. A predicted value corresponds to the predicted rating given to a movie by a user based on his preferences.

In [ ]:
# Create the randomized weights

# Number of users : 943
N = len(Ratings)

# Number of movies : 1682
M = len(Ratings[0])

# Num of Features
K = 10

 
Theta = np.random.rand(N,K)
X = np.random.rand(M,K)
print(X, Theta)

<h3>Suitable parameters are chosen to improve accuracy. This needs to be worked on to get improved accuracy.</h3>

In [ ]:
# Decide parameters
iterations = 50
alpha = 0.02
lambdaa = 0.02

<h3>Gradient descent is performed so as to train the user and movie feature vectors</h3>

Collaborative filtering algorithm is used for this purpose.

<h3> The following cost function is used which simultenously minimise both movie and user vectors.</h3>

![](J.png)

<h3>The weights are adjusted in every iteration of gradient descent as shown.</h3>

![](G.png)

In [ ]:
# Gradient descent
X = X.T
alpha_0 = alpha
J_history = [0 for i in range(iterations)]
for itr in range(iterations):
    alpha = alpha_0 / (1 + (itr) / 20)
    for i in range(N):
        for j in range(M):
            if Ratings[i][j] > 0:
                error = np.dot(Theta[i,:], X[:,j])
                for k in range(K):
                    Theta[i][k] = Theta[i][k] - alpha * ((error - Ratings[i][j]) * X[k][i]   + lambdaa * Theta[i][k])
                    X[k][j] = X[k][j] - alpha * ((error - Ratings[i][j]) * Theta[i][k]   + lambdaa * X[k][j])

    J = 0
    for i in range(N):
        for j in range(M):
            if Ratings[i][j] > 0:

                J = J + 0.5 * pow(Ratings[i][j] - np.dot(Theta[i,:],X[:,j]), 2)

                for k in range(K):

                    J = J + 0.5 * (lambdaa / 2) * (pow(Theta[i][k],2) + pow(X[k][j],2))

    print("In iteration ", itr, " cost function is: ", J)
    J_history[itr] = J

    if J < 0.001:

        break



<h2>The gradient descent plot is shown below</h2>

![Gradient Descent](output.png)

In [ ]:
# Plot the cost function
import matplotlib.pyplot as plt
plt.plot(range(len(J_history)), J_history, 'r')
plt.show()

In [ ]:
print(np.dot(Theta, X))

<h3> Predicted ratings of all the movies for user 1</h3>

In [ ]:
H = np.dot(Theta, X)

# Predict movies like by user1
user_1 = H[1,:]
user_1 = np.sort(user_1)
user_1[:: -1]